In [214]:
!mkdir dataset
!curl -L -o dataset/energy_data.csv "https://raw.githubusercontent.com/yixiangD/AccurateBG/main/time-gan/data/energy_data.csv" && ls -l dataset/energy_data.csv
!curl -L -o dataset/stock_data.csv "https://raw.githubusercontent.com/yixiangD/AccurateBG/refs/heads/main/time-gan/data/stock_data.csv" && ls -l dataset/stock_data.csv

A subdirectory or file dataset already exists.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 43 4299k   43 1873k    0     0  4133k      0  0:00:01 --:--:--  0:00:01 4152k
100 4299k  100 4299k    0     0  7282k      0 --:--:-- --:--:-- --:--:-- 7299k
'ls' is not recognized as an internal or external command,
operable program or batch file.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  222k  100  222k    0     0   766k      0 --:--:-- --:--:-- --:--:--  774k
'ls' is not recognized as an internal or external command,
operable program or batch file.


In [215]:
import pandas as pd
import math

In [216]:
PATH_ENERGY = 'dataset/energy_data.csv'
PATH_STOCK = 'dataset/stock_data.csv'

In [217]:
data_energy = pd.read_csv(PATH_ENERGY)
data_stock = pd.read_csv(PATH_STOCK)

In [218]:
data_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3685 entries, 0 to 3684
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3685 non-null   float64
 1   High       3685 non-null   float64
 2   Low        3685 non-null   float64
 3   Close      3685 non-null   float64
 4   Adj_Close  3685 non-null   float64
 5   Volume     3685 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 172.9 KB


In [219]:
data_energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Appliances   19735 non-null  int64  
 1   lights       19735 non-null  int64  
 2   T1           19735 non-null  float64
 3   RH_1         19735 non-null  float64
 4   T2           19735 non-null  float64
 5   RH_2         19735 non-null  float64
 6   T3           19735 non-null  float64
 7   RH_3         19735 non-null  float64
 8   T4           19735 non-null  float64
 9   RH_4         19735 non-null  float64
 10  T5           19735 non-null  float64
 11  RH_5         19735 non-null  float64
 12  T6           19735 non-null  float64
 13  RH_6         19735 non-null  float64
 14  T7           19735 non-null  float64
 15  RH_7         19735 non-null  float64
 16  T8           19735 non-null  float64
 17  RH_8         19735 non-null  float64
 18  T9           19735 non-null  float64
 19  RH_9

In [220]:
data_energy.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [221]:
data_stock.head()

,Open,High,Low,Close,Adj_Close,Volume
0,49.676899,51.693783,47.669952,49.845802,49.845802,44994500
1,50.178635,54.187561,49.925285,53.805050,53.805050,23005800
2,55.017166,56.373344,54.172661,54.346527,54.346527,18393200
3,55.260582,55.439419,51.450363,52.096165,52.096165,15361800
4,52.140873,53.651051,51.604362,52.657513,52.657513,9257400


In [222]:
colunas = data_energy.columns

In [223]:
split_idx = int(len(data_energy) * 0.7)
train = data_energy.iloc[:split_idx].reset_index(drop=True)
test = data_energy.iloc[split_idx:].reset_index(drop=True)
print(f"train: {len(train)} rows, test: {len(test)} rows")

train: 13814 rows, test: 5921 rows


In [224]:
x_train = train[colunas[0]]
y_train = train[colunas[1]]

x_test = test[colunas[0]]
y_test = test[colunas[1]]

In [225]:
def calculo_b1_b0(column_x,column_y):
    x_mean = column_x.sum() / len(column_x)
    y_mean = column_y.sum() / len(column_y)
    
    numerador = 0
    denominador = 0
    
    for i in range(len(column_x)):
        numerador += (column_x[i] - x_mean) * (column_y[i] - y_mean)
        denominador += (column_x[i] - x_mean) ** 2
    
    b1 = numerador / denominador
    
    b0 = y_mean - (x_mean*b1)
    
    return b1,b0
b1,b0 = calculo_b1_b0(x_train,y_train)
b1,b0

(0.01818574059370359, 2.7556045080862406)

In [226]:
def calculo_pred(b1,b0,x):
    return (b1 * x) + b0

In [227]:
def pred_test(column_x_test):
    column_y_pred = []
    
    for x in column_x_test:
        column_y_pred.append(calculo_pred(b1,b0,x))
    
    return column_y_pred
column_y_pred = pred_test(x_test)

In [228]:
def rmse(column_y_real,column_y_pred):
    n = len(column_y_real)
    numerador = 0.0
    
    for i in range(n):
        numerador += (column_y_real[i] - column_y_pred[i]) ** 2
    
    result = numerador / n
    
    return math.sqrt(result)

rmse(y_test,column_y_pred)

6.277822495848264

In [229]:
def mse(column_y_real,column_y_pred):
    n = len(column_y_real)
    numerador = 0.0
    
    for i in range(n):
        numerador += (column_y_real[i] - column_y_pred[i]) ** 2
    
    result = numerador / n
    
    return result

mse(y_test,column_y_pred)

39.41105528937853